In [1]:
import os
import sys

sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib as plt
import sklearn
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df = pd.read_hdf("../data_extract/features_dataset_2b.h5")

In [2]:
df = df[np.isfinite(df['centroid'])]
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['id'] = le.fit_transform(df['pid'])


In [3]:
from numpy import zeros
def gfit(chlist, xtrain, id_range):
    le = LabelEncoder()
    mm = list()
    covm = list()
    for pid in range(id_range):
        mm.append(np.mean(xtrain[chlist].loc[df['id']==pid],axis=0))
        covm.append(np.cov(xtrain[chlist].loc[df['id']==pid], rowvar=False))
    return mm, covm


def predict(chlist,xtest,mm,covm,y_test):
    true_results = 0
    cls = len(mm)
    total = y_test.count()
    dist = zeros((total,cls))
    from scipy.stats import multivariate_normal
    for i in range(cls):
        dist[:,i] = multivariate_normal.pdf(xtest, mean=mm[i], cov=covm[i],allow_singular=True)
#     print(y_test.tolist()[0])
    y_test = y_test.tolist()
    for i in range(total):
        probs = dist[i,:]
        if probs[y_test[i]] == max(probs):
            true_results = true_results + 1
    print(true_results,total)
    
    
def train_MLE(df):
    chlist = ["chroma{}".format(i) for i in range(12)]
    
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(
        df[chlist], df["id"], test_size=0.3, random_state=42)
    
    x_train['id'] = y_train
    
    mm, covm = gfit(chlist,x_train,40)
    predict(chlist,x_test,mm,covm,y_test)

In [4]:
train_MLE(df)

(4550, 44997)
